In [ ]:
import os # Configure which GPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
import random
import torch

from utils.NN_utils import BeamPredictionModel, BlockPredictionModel, BestGainPredictionModel
from utils.NN_utils import BeamPredictionLSTMModel, BlockPredictionLSTMModel, BestGainPredictionLSTMModel
from utils.NN_utils import preprocess_data
from utils.options import args_parser
from utils.mox_utils import setup_seed, get_save_dirs, np2torch, save_NN_results
from utils.data_utils import get_prepared_dataset, prepare_dataset
from utils.beam_utils import generate_dft_codebook, beamIdPair_to_beamPairId, beamPairId_to_beamIdPair



In [ ]:
setup_seed(20)
freq = 28e9
DS_start, DS_end = 800, 900
preprocess_mode = 2
pos_in_data = preprocess_mode==2
look_ahead_len = 1
n_pilot = 16
M_r, N_bs, M_t = 8, 4, 64
P_t = 1e-1
P_noise = 1e-14 # -174dBm/Hz * 1.8MHz = 7.165929069962946e-15 W
gpu = 4
device = f'cuda:{gpu}' if torch.cuda.is_available() else 'cpu'
print('Using device: ', device)

prepared_dataset_filename, data_np, veh_h_np, veh_pos_np, best_beam_pair_index_np \
    = get_prepared_dataset(preprocess_mode, DS_start, DS_end, M_t, M_r, freq, n_pilot, N_bs, P_t, P_noise, look_ahead_len)
data_torch = np2torch(data_np[:,0,...],device) 
veh_h_torch = np2torch(veh_h_np[:,-1,...],device) 
veh_pos_torch = np2torch(veh_pos_np[:,-1,...],device) 
best_beam_pair_index_torch = np2torch(best_beam_pair_index_np[:,-1,...],device)
block_labels = (veh_h_torch[:,0,:,0]==0).long().to(device)
DFT_tx = generate_dft_codebook(M_t)
DFT_rx = generate_dft_codebook(M_r)
beamPairId = best_beam_pair_index_torch.detach().cpu().numpy()
beamIdPair = beamPairId_to_beamIdPair(beamPairId,M_t,M_r)
num_car, _, num_bs, _ = veh_h_torch.shape
channel = veh_h_torch.detach().cpu().numpy()
g_opt = np.zeros((num_car,num_bs)).astype(np.float32)
for veh in range(num_car):
    for bs in range(num_bs):
        g_opt[veh,bs] = 1/np.sqrt(M_t*M_r)*np.abs(np.matmul(np.matmul(channel[veh,:,bs,:], DFT_tx[:,beamIdPair[veh,bs,0]]).T.conjugate(),DFT_rx[:,beamIdPair[veh,bs,1]]))
        g_opt[veh,bs] = 20 * np.log10(g_opt[veh,bs] + 1e-9) 
g_opt_normalized = g_opt / 20 + 7
gain_labels = torch.tensor(g_opt_normalized).to(device)

feature_input_dim = data_torch.shape[-1] * 2 - pos_in_data * 2
num_bs = 4
num_beampair = M_t * M_r

beam_pred_model = BeamPredictionModel(feature_input_dim, num_bs, num_beampair).to(device)
block_pred_model = BlockPredictionModel(feature_input_dim, num_bs).to(device)
gain_pred_model = BestGainPredictionModel(feature_input_dim, num_bs).to(device)

beam_pred_model.load_state_dict(torch.load('NN_result/400_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead1/models/beampred_valAcc87.67%_2025-04-22_17:45:51.pth'))
block_pred_model.load_state_dict(torch.load('NN_result/400_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead1/models/blockpred_valAcc98.03%_2025-04-22_17:59:06.pth'))
gain_pred_model.load_state_dict(torch.load('NN_result/400_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead1/models/gainpred_valMae2.50dB_2025-04-22_18:02:25.pth'))

data = preprocess_data(data_torch, pos_in_data)
beam_pred_model.eval()
block_pred_model.eval()
gain_pred_model.eval()
with torch.no_grad():
    beam_pred = beam_pred_model(data).argmax(dim=-1)
    block_pred = block_pred_model(data).argmax(dim=-1)
    gain_pred = gain_pred_model.pred(data)


In [ ]:
(block_pred == block_labels).to(torch.float).mean()

In [ ]:
np.abs((gain_pred.detach().cpu().numpy() - g_opt)).mean()

In [ ]:
gain_pred.max(), gain_pred.min(), g_opt.max(), g_opt.min() #  -69.123604~-148.2894

In [ ]:
# gain_pred and block_pred coordination
np.abs(((gain_pred*(1-block_pred) - 180.*block_pred).detach().cpu().numpy() - g_opt)).mean()

In [ ]:
# Clip gain_pred
gain_pred_clipLB = gain_pred*(gain_pred>=-180.) - 180.*(gain_pred<-180.)
np.abs((gain_pred_clipLB.detach().cpu().numpy() - g_opt)).mean()

# LSTM

In [ ]:
setup_seed(20)
freq = 28e9
DS_start, DS_end = 800, 900
preprocess_mode = 2
pos_in_data = preprocess_mode==2
look_ahead_len = 3
n_pilot = 16
M_r, N_bs, M_t = 8, 4, 64
P_t = 1e-1
P_noise = 1e-14 # -174dBm/Hz * 1.8MHz = 7.165929069962946e-15 W
gpu = 4
device = f'cuda:{gpu}' if torch.cuda.is_available() else 'cpu'
print('Using device: ', device)

prepared_dataset_filename, data_np, veh_h_np, veh_pos_np, best_beam_pair_index_np \
    = get_prepared_dataset(preprocess_mode, DS_start, DS_end, M_t, M_r, freq, n_pilot, N_bs, P_t, P_noise, look_ahead_len)
data_torch = np2torch(data_np[:,:-1,...],device) 
veh_h_torch = np2torch(veh_h_np[:,-1,...],device) 
veh_pos_torch = np2torch(veh_pos_np[:,-1,...],device) 
best_beam_pair_index_torch = np2torch(best_beam_pair_index_np[:,-1,...],device)
block_labels = (veh_h_torch[:,0,:,0]==0).long().to(device)
DFT_tx = generate_dft_codebook(M_t)
DFT_rx = generate_dft_codebook(M_r)
beamPairId = best_beam_pair_index_torch.detach().cpu().numpy()
beamIdPair = beamPairId_to_beamIdPair(beamPairId,M_t,M_r)
num_car, _, num_bs, _ = veh_h_torch.shape
channel = veh_h_torch.detach().cpu().numpy()
g_opt = np.zeros((num_car,num_bs)).astype(np.float32)
for veh in range(num_car):
    for bs in range(num_bs):
        g_opt[veh,bs] = 1/np.sqrt(M_t*M_r)*np.abs(np.matmul(np.matmul(channel[veh,:,bs,:], DFT_tx[:,beamIdPair[veh,bs,0]]).T.conjugate(),DFT_rx[:,beamIdPair[veh,bs,1]]))
        g_opt[veh,bs] = 20 * np.log10(g_opt[veh,bs] + 1e-9) 
g_opt_normalized = g_opt / 20 + 7
gain_labels = torch.tensor(g_opt_normalized).to(device)

feature_input_dim = data_torch.shape[-1] * 2 - pos_in_data * 2
num_bs = 4
num_beampair = M_t * M_r

beam_pred_lstm_model = BeamPredictionLSTMModel(feature_input_dim, num_bs, num_beampair).to(device)
block_pred_lstm_model = BlockPredictionLSTMModel(feature_input_dim, num_bs).to(device)
gain_pred_lstm_model = BestGainPredictionLSTMModel(feature_input_dim, num_bs).to(device)

beam_pred_lstm_model.load_state_dict(torch.load('NN_result/400_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead3/models/beampred_lstm_valAcc87.48%_2025-04-22_16:58:35.pth'))
block_pred_lstm_model.load_state_dict(torch.load('NN_result/400_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead3/models/blockpred_lstm_valAcc98.35%_2025-04-22_16:58:42.pth'))
gain_pred_lstm_model.load_state_dict(torch.load('NN_result/400_800_3Dbeam_tx(1,64)_rx(1,8)_freq2.8e+10_Np16_mode2_lookahead3/models/gainpred_lstm_valMae2.51dB_2025-04-22_17:03:25.pth'))

data = preprocess_data(data_torch, pos_in_data)
beam_pred_model.eval()
block_pred_model.eval()
gain_pred_model.eval()
with torch.no_grad():
    beam_pred = beam_pred_lstm_model(data).argmax(dim=-1)
    block_pred = block_pred_lstm_model(data).argmax(dim=-1)
    gain_pred = gain_pred_lstm_model.pred(data)


In [ ]:
(block_pred == block_labels).to(torch.float).mean()

In [ ]:
np.abs((gain_pred.detach().cpu().numpy() - g_opt)).mean()

In [ ]:
gain_pred.max(), gain_pred.min(), g_opt.max(), g_opt.min() #  -69.123604~-148.2894

In [ ]:
# gain_pred and block_pred coordination
np.abs(((gain_pred*(1-block_pred) - 180.*block_pred).detach().cpu().numpy() - g_opt)).mean()

In [ ]:
# Clip gain_pred
gain_pred_clipLB = gain_pred*(gain_pred>=-180.) - 180.*(gain_pred<-180.)
np.abs((gain_pred_clipLB.detach().cpu().numpy() - g_opt)).mean()

In [ ]:
(gain_pred.detach().cpu().numpy()-g_opt).reshape(-1)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.hist((gain_pred.detach().cpu().numpy()-g_opt).reshape(-1), bins=100)
plt.show()

In [ ]:
block_pred.shape, gain_pred.shape

In [ ]:
block_pred_lstm_model(data).shape, gain_pred_lstm_model.pred(data).shape

In [ ]:
output_block_pred_lstm_model = block_pred_lstm_model(data)
output_gain_pred_lstm_model = gain_pred_lstm_model(data)

In [ ]:
output_block_pred_lstm_model.shape, output_block_pred_lstm_model

In [ ]:
output_gain_pred_lstm_model.shape, output_gain_pred_lstm_model

In [ ]:
torch.nn.functional.softmax(output_block_pred_lstm_model, dim=-1).argmax(dim=-1).shape, torch.nn.functional.softmax(output_block_pred_lstm_model, dim=-1).argmax(dim=-1)

In [ ]:
error_loc = ~(block_labels == torch.nn.functional.softmax(output_block_pred_lstm_model, dim=-1).argmax(dim=-1))

In [1]:
import os # Configure which GPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
import random
import torch

# from utils.NN_utils import BeamPredictionModel, train_beampred_lstm_model
from utils.NN_utils import BeamPredictionModel, train_beampred_lstm_model
from utils.options import args_parser
from utils.mox_utils import setup_seed, get_save_dirs, save_NN_results
from utils.plot_utils import plot_record_metrics
from utils.data_utils import get_prepared_dataset, prepare_dataset


# 设置随机数种子
setup_seed(20)
freq = 28e9
DS_start, DS_end = 400, 800
preprocess_mode = 2
look_ahead_len = 3
n_pilot = 16
M_r, N_bs, M_t = 8, 4, 64
P_t = 1e-1
P_noise = 1e-14 # -174dBm/Hz * 1.8MHz = 7.165929069962946e-15 W
gpu = 7
device = f'cuda:{gpu}' if torch.cuda.is_available() else 'cpu'
print('Using device: ', device)

prepared_dataset_filename, data_np, veh_h_np, veh_pos_np, best_beam_pair_index_np \
    = get_prepared_dataset(preprocess_mode, DS_start, DS_end, M_t, M_r, freq, n_pilot, N_bs, P_t, P_noise, look_ahead_len)
data_torch = torch.tensor(data_np[:,:-1,...]) 
veh_h_torch = torch.tensor(veh_h_np[:,-1,...]) 
veh_pos_torch = torch.tensor(veh_pos_np[:,-1,...]) 
best_beam_pair_index_torch = torch.tensor(best_beam_pair_index_np[:,-1,...])

# 数据增强 将原长度为look_ahead_len的序列进行截断处理
lengths = [look_ahead_len]*len(data_np)
for input_seq_len in range(look_ahead_len-1, 0, -1):
    # input_seq_len 2,1
    data_np_clipped = np.zeros_like(data_np[:,:-1,...])
    data_np_clipped[:,:input_seq_len,...] = data_np[:,-input_seq_len-1:-1,...]
    data_torch = torch.concat((data_torch, torch.tensor(data_np_clipped)), dim=0)
    lengths.extend([input_seq_len]*len(data_np_clipped))
    veh_h_torch = torch.concat((veh_h_torch, torch.tensor(veh_h_np[:,-1,...])), dim=0)
    veh_pos_torch = torch.concat((veh_pos_torch, torch.tensor(veh_pos_np[:,-1,...])), dim=0)
    best_beam_pair_index_torch = torch.concat((best_beam_pair_index_torch, torch.tensor(best_beam_pair_index_np[:,-1,...])), dim=0)


Using device:  cuda:7


In [2]:
data_torch_cut = data_torch[::10000,...]
lengths_cut = np.array(lengths)[::10000,...]
pack_padded_data_torch = torch.nn.utils.rnn.pack_padded_sequence(data_torch_cut, lengths_cut, batch_first=True, enforce_sorted=False)

In [3]:
from utils.NN_utils import BeamPredictionLSTMModel

lstm_model = torch.nn.LSTM(
            input_size=130,
            hidden_size=32,
            num_layers=1,
            batch_first=True,
        ).to(device)


packed_out, hidden =  lstm_model(pack_padded_data_torch.to(torch.float).to(device))
(PackedSequence, batch_sizes, sorted_indices, unsorted_indices) = packed_out

/home/ubuntu/anaconda3/envs/sionna/lib/python3.10/site-packages/torch/nn/utils/rnn.py:155: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at /pytorch/aten/src/ATen/native/Copy.cpp:308.)
  data = self.data.to(*args, **kwargs)


In [4]:
hidden[0].shape, hidden[1].shape

(torch.Size([1, 105, 32]), torch.Size([1, 105, 32]))

In [5]:
PackedSequence.shape, batch_sizes

(torch.Size([210, 32]), tensor([105,  70,  35]))

In [6]:
out_unpacked, valid_lengths = torch.nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)
        

In [7]:
out_unpacked.shape, out_unpacked

(torch.Size([105, 3, 32]),
 tensor([[[ 9.4247e-02,  7.4599e-11, -1.7691e-22,  ..., -1.6173e-04,
            1.7199e-02, -1.4099e-02],
          [ 9.9912e-02,  9.3010e-11, -1.5589e-22,  ..., -1.2791e-04,
            1.6201e-02, -1.3515e-02],
          [ 8.7493e-02,  1.1500e-10, -1.4372e-22,  ..., -1.2990e-04,
            2.0431e-02, -1.6299e-02]],
 
         [[-2.5680e-09, -2.9711e-26, -7.6159e-01,  ...,  3.5282e-04,
           -9.4532e-34,  9.3365e-34],
          [-4.6121e-09, -2.5454e-26, -7.6159e-01,  ...,  5.4317e-04,
           -1.0317e-33,  1.2130e-33],
          [-6.0852e-09, -3.9183e-26, -7.6159e-01,  ...,  6.4369e-04,
           -1.9368e-33,  2.4568e-33]],
 
         [[-5.0990e-16,  9.4696e-17,  2.7420e-12,  ...,  5.9959e-17,
           -1.1768e-05, -4.5151e-15],
          [-7.7781e-16,  1.2268e-16,  4.9919e-12,  ...,  6.0626e-17,
           -2.2044e-05, -1.0320e-14],
          [-1.2402e-15,  1.5581e-16,  7.1382e-12,  ...,  7.0224e-17,
           -3.2520e-05, -1.7561e-14]],
 
 

In [15]:
ddd = out_unpacked[torch.arange(out_unpacked.shape[0]), valid_lengths-1, ...]

In [16]:
out_unpacked[100], ddd[100]

(tensor([[ 0.0000e+00,  7.6159e-01,  1.2331e-43,  7.6159e-01,  0.0000e+00,
          -0.0000e+00,  2.0207e-26, -0.0000e+00, -2.1578e-15,  1.6910e-23,
          -7.6159e-01,  1.3482e-09,  0.0000e+00,  1.1225e-36,  0.0000e+00,
           3.1947e-17,  1.9552e-17,  3.0804e-21,  7.6159e-01,  2.8026e-45,
           1.3438e-24,  1.1405e-13,  7.5342e-01, -1.0125e-01, -7.6159e-01,
          -1.5728e-19, -0.0000e+00, -6.5850e-01, -3.8912e-16, -7.5832e-01,
           7.6159e-01, -7.6159e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.

AttributeError: 'Tensor' object has no attribute 'dims'

In [25]:
torch.tensor([1,2,3]) is not False

True